In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

***

In [2]:
%%time
train = pd.read_parquet("../data/ext/amex-data-integer-dtypes-parquet-format/train.parquet")
test = pd.read_parquet("../data/ext/amex-data-integer-dtypes-parquet-format/test.parquet")

CPU times: user 31.7 s, sys: 15.8 s, total: 47.5 s
Wall time: 9.49 s


In [3]:
train["S_2"] = pd.to_datetime(train["S_2"])
test["S_2"] = pd.to_datetime(test["S_2"])

***
## identify columns that are constant over time

In [4]:
constant_cols = list()

for col in tqdm(train.columns[2:]):
    if train.groupby("customer_ID")[col].nunique().max() == 1:
        print(col)
        constant_cols.append(col)

100% 188/188 [05:30<00:00,  1.76s/it]


In [5]:
# there are no such columns ...
constant_cols

[]

***
## convert to wide format

In [6]:
# create timestep variable

train["timestep"] = 14 - train.groupby("customer_ID")["S_2"].rank(ascending=False).astype(int)
test["timestep"] = 14 - test.groupby("customer_ID")["S_2"].rank(ascending=False).astype(int)

In [7]:
# pivot the dataframe col by col

all_dfs = list()

for col in tqdm(train.columns[2:]):
    df = train.pivot(index="customer_ID", columns="timestep", values=col)
    df.columns = [f"{col}-{i}" for i in df.columns]
    all_dfs.append(df)


100% 189/189 [06:59<00:00,  2.22s/it]


In [ ]:
# todo: concatenate by axis=0
r = pd.concat(all_dfs)

In [ ]:
r

***